In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
#parameters
P0=101325.5#地上大気圧力
T0=288.15#地上大気温度
Ma=0.0#巡航速度
m_dot0=1.0#空気流量
gamma_c=1.4
omega_1=0.02#インテーク圧損
Cp_c=1004
omega_np=0.02#ファンノズル圧損
eta_fan=0.9#fan効率
eta_clp=0.9#低圧圧縮機効率
eta_chp=0.9#高圧圧縮機効率
omega_cb=0.05#燃焼圧損
eta_cb=0.99#燃焼効率
Cp_t=1150
gamma_t=1.33
h=36.7*10**6#低位発熱量
eta_tlp=0.9#低圧タービン効率
eta_thp=0.9#高圧タービン効率
omega_core=0.05#ノズル圧損

#抽気割合
alpha=0.2
beta1=alpha*0.75#高圧タービン抽気割合
beta2=alpha-beta1#低圧タービン抽気割合

R=287#気体定数
gc=9.8#重力定数
eta_mac=0.99#機械効率

#必要推力
TN=30000
HTR=0.3#チップハブ比


In [254]:
class jetengine(object):
    
    def __init__(self,BPR,FPR,OPR,TIT):
        self.BPR=BPR
        self.FPR=FPR
        self.OPR=OPR
        self.TIT=TIT
    
    
    """設計点計算部分負荷計算用 地上設計点"""
    def cycle(self,M,H):
        pai_clp=np.sqrt(self.OPR)
        pai_chp=np.sqrt(self.OPR)
        #入口大気
        Pt0=P0*(1.0+(gamma_c-1.0)/2.0*Ma**2)**((gamma_c)/(gamma_c-1))
        Tt0=T0*(1.0+(gamma_c-1.0)/2.0*Ma**2)
        V0=np.sqrt(gamma_c*R*T0)

        #インテーク入口
        Pt2=Pt0*(1.0-omega_1)
        Tt2=Tt0
        m_dot2=m_dot0
        I2=Cp_c*Tt2

        #ファン出口(バイパス)
        self.Pt13=self.FPR*Pt2
        self.Tt13=Tt2*(1.0+(self.FPR**((gamma_c-1.0)/gamma_c)-1.0)/eta_fan)
        I13=I2+Cp_c*(self.Tt13-Tt2)
        self.m_dot13=self.BPR/(1.0+self.BPR)*m_dot0

        #ファンノズル
        Pt19=self.Pt13*(1.0-omega_np)
        Tt19=self.Tt13


        #ファン出口(コア)
        self.Pt21=self.FPR*Pt2
        self.Tt21=Tt2*(1.0+(self.FPR**((gamma_c-1.0)/gamma_c)-1.0)/eta_fan)
        I21=I2+Cp_c*(self.Tt21-Tt2)
        m_dot21=1.0/(1.0+self.BPR)*m_dot0
        

        #低圧圧縮機出口
        self.Pt25=self.Pt21*pai_clp
        self.Tt25=self.Tt21*(1.0+(pai_clp**((gamma_c-1.0)/gamma_c)-1.0)/eta_clp)
        I25=Cp_c*self.Tt25
        self.m_dot25=m_dot21
        M25=0.4#軸流マッハ数
        P25=self.Pt25/(1.0+(gamma_c-1.0)/2.0*M25**2)**(gamma_c/(gamma_c-1.0))
        T25=self.Tt25/(1.0+(gamma_c-1.0)/2.0*M25**2)
        rou25=P25/(R*T25)
        V25=M25*np.sqrt(gamma_c*R*T25)
        self.A25=self.m_dot25/(rou25*V25)#面積
        

        #高圧圧縮機出口
        self.Pt3=self.Pt25*pai_chp
        self.Tt3=self.Tt25*(1.0+(pai_chp**((gamma_c-1.0)/gamma_c)-1.0)/eta_chp)
        I3=I25+Cp_c*(self.Tt3-self.Tt25)
        self.m_dot3=self.m_dot25
        M3=0.4#軸流マッハ数
        P3=self.Pt3/(1.0+(gamma_c-1.0)/2.0*M3**2)**(gamma_c/(gamma_c-1.0))
        T3=self.Tt3/(1.0+(gamma_c-1.0)/2.0*M3**2)
        rou3=P3/(R*T3)
        V3=M3*np.sqrt(gamma_c*R*T3)
        self.A3=self.m_dot3/(rou3*V3)#面積
        

        #燃焼器出口
        Pt4=self.Pt3*(1.0-omega_cb)
        Tt4=self.TIT
        I4=I3+Cp_t*(Tt4-self.Tt3)
        self.m_dot_fuel=(self.m_dot3-alpha*self.m_dot3)*(I4-I3)/(eta_cb*h-I4)#エネルギー保存
        m_dot4=self.m_dot3-alpha*self.m_dot3+self.m_dot_fuel

        #高圧タービン出口
        I_d45=I4-self.m_dot3*(I3-I25)/(eta_mac*m_dot4)
        T_d45=Tt4-(I4-I_d45)/Cp_t
        Pt45=Pt4*((T_d45/Tt4-1.0)/eta_thp+1.0)**(gamma_t/(gamma_t-1.0))
        self.I45=(m_dot4*I_d45+self.m_dot3*beta1*I3)/(m_dot4+self.m_dot3*beta1)
        Tt45=Tt4-(I4-self.I45)/Cp_t
        m_dot45=m_dot4+beta1*self.m_dot3
        self.M45=0.4#高圧タービン出口マッハ数

        #低圧タービン出口
        I_d5=self.I45-(self.BPR/(1.0+self.BPR)*m_dot2*(I13-I2)+1.0/(1.0+self.BPR)*m_dot2*(I25-I2))/(eta_tlp*m_dot45)
        T_d5=Tt45-(I_d5-self.I45)/Cp_t
        Pt5=Pt45*((T_d5/Tt45-1.0)/eta_tlp+1.0)**(gamma_t/(gamma_t-1.0))
        I5=(m_dot45*I_d5+self.m_dot3*beta2*I3)/(m_dot45+self.m_dot3*beta2)
        Tt5=Tt45-(I5-self.I45)/Cp_t
        m_dot5=m_dot45+self.m_dot3*beta2
        self.M5=0.3#低圧タービン出口マッハ数

        #ノズル
        Pt9=Pt5*(1.0-omega_core)
        Tt9=Tt5
        m_dot9=m_dot5



        if Pt9/Pt0<=((gamma_t+1.0)/2.0)**(gamma_t/(gamma_t-1)):
            P9=P0
            M9=np.sqrt(2.0/(gamma_t-1.0)*((Pt9/P9)**((gamma_t-1.0)/gamma_t)-1.0))
            T9=Tt9/(1.0+(gamma_t-1.0)/2.0*M9**2)
            rou9=P9/(R*T9)
            V9=np.sqrt(gamma_t*R*T9)*M9
            self.A9=m_dot9/(rou9*V9)
        else:
            M9=1.0
            T9=Tt9/(gamma_t+1.0)/2.0
            P9=Pt9/((gamma_t+1.0)/2.0)**(gamma_t/(gamma_t-1.0))
            rou9=P9/(R*T9)
            V9=np.sqrt(gamma_t*R*T9)*M9
            self.A9=m_dot9/(rou9*V9)

        m_dot19=self.m_dot13
        P19=P0
        M19=np.sqrt(2.0/(gamma_c-1.0)*((Pt19/P19)**((gamma_c-1.0)/gamma_c)-1.0))
        T19=Tt19/(1.0+(gamma_c-1.0)/2.0*M19**2)
        V19=np.sqrt(gamma_c*R*T19)*M19
        rou19=P19/(R*T19)
        self.A19=m_dot19/(rou19*V19)




        #推力
        FN=(m_dot19*V19+m_dot9*V9-m_dot0*V0)/gc+self.A19*(P19-P0)+self.A9*(P9-P0)
        #SFC[kg/s]
        SFC=self.m_dot_fuel/FN*3600
        #比推力
        ISP=FN/m_dot0
        #入口出口圧力比
        tau=T9/T0
        
        """部分負荷計算"""
        """飛行条件
           M:飛行マッハ数
           H:高度"""
        Tref=288.15
        Pref=101325
        #圧縮機マップ近似
        a=5.0
        b=10.0
        
        #入口大気
        if H<11000.0:
            Tref=Tref-6.5*(H/1000.0)
        elif 11000.0<=H<20000.0:
            Tref=Tref-6.5*(11000.0/1000.0)
        elif 20000.0<=H:
            Tref=Tref-6.5*(11000.0/1000.0)+1.0*(H-20000.0)/1000.0
            
        if H<11000.0:
            Pref=Pref*(1.0-6.8753e-6*H/0.3048)**(5.2561)
        else:
            Pref=Pref*0.22336*np.exp(-(H/0.3048-3.6089e4)/2.086e4)
            
        Tref=Tref*(1.0+(gamma_c-1.0)/2.0*M**2)
        Pref=Pref*(1.0+(gamma_c-1.0)/2.0*M**2)**(gamma_c/(gamma_c-1.0))
        
        
        LP_rate=0.8#計算? LP軸修正回転数割合
        
        #loop1
        
        #基準値
        """ファンのマッハ数はM21"""
        m_fan_fix=(self.m_dot13*np.sqrt(self.Tt13/Tref)/(self.Pt13/Pref))#修正流量
        MFP_ref_fan=m_fan_fix*np.sqrt(Tref)/(self.A19*Pref)
        pai_ref_fan=self.Pt13/Pref
        
        MFP_rate_max=1.2
        MFP_rate_min=0.4
        MFP_rate_step=0.01
        MFP_rate=MFP_rate_max#修正流量の割合
        eps_area_list=[]#面積残差リスト
        for i in range(0,int((MFP_rate_max-MFP_rate_min)/MFP_rate_step),1):
            """fan"""
            pai_fan=1.0+(2.0-(MFP_rate*1.0/(LP_rate))**b)**(1.0/a)*LP_rate**2*(pai_ref_fan-1.0)#全圧比もとまる
            tau_fan=(1.0+(pai_fan**((gamma_c-1.0)/gamma_c)-1.0)/eta_fan)
            mfan=pai_fan/np.sqrt(tau_fan)*MFP_rate*m_fan_fix#ファンバイパス流量
            mcore=m_dot0-mfan#コア流量
            
            Ptfan=Pref*pai_fan#ファンの全圧
            Ttfan=Tref*tau_fan#ファンの全温
            Ifan=Cp_c*Ttfan#ファン出口のエンタルピー
            
            """LPC"""
            """LPCのマッハ数はM25"""
            m_lpc_fix=(self.m_dot25*np.sqrt(self.Tt25/Tref)/(self.Pt25/Pref))#修正流量
            MFP_ref_lpc=m_lpc_fix*np.sqrt(Tref)/(self.A25*Pref)
            pai_ref_lpc=self.Pt25/Pref
            pai_lpc=1.0+(2.0-(MFP_rate*1.0/((LP_rate)))**b)**(1.0/a)*LP_rate**2*(pai_ref_lpc-1.0)#全圧比もとまる
            tau_lpc=(1.0+(pai_lpc**((gamma_c-1.0)/gamma_c)-1.0)/eta_clp)
            m_lpc=mcore
            Ptlpc=Pref*pai_lpc
            Ttlpc=Tref*tau_lpc
            
            
            #loop2
            """HPC"""
            """HPCのマッハ数はM3"""
            m_hpc_fix=(self.m_dot3*np.sqrt(self.Tt3/Tref)/(self.Pt3/Pref))
            MFP_ref_hpc=m_hpc_fix*np.sqrt(Tref)/(self.A3*Pref)
            pai_ref_hpc=self.Pt3/Pref
            eps_ene_list=[]#エネルギー残差リスト
            HP_rate_step=0.01
            rate_max=1.0
            rate_min=0.5
            HP_rate=rate_max#HP軸の修正回転数を仮定する
            for i in range(0,int((rate_max-rate_min)/HP_rate_step),1):
                pai_hpc=1.0+(2.0-(MFP_rate*1.0/((HP_rate)))**b)**(1.0/a)*HP_rate**2*(pai_ref_hpc-1.0)#全圧比もとまる
                tau_hpc=(1.0+(pai_hpc**((gamma_c-1.0)/gamma_c)-1.0)/eta_chp)
                Pthpc=Pref*pai_hpc
                Tthpc=Tref*tau_hpc
                m_hpc=m_lpc#m3
                Ihpc=Cp_c*Tthpc#I3
                
                #燃焼器
                """CC"""
                m_fuel=self.m_dot_fuel#燃料流量
                Icb=(m_fuel*eta_cb*h+m_hpc*(1.0-alpha)*Ihpc)/(m_fuel+m_hpc*(1.0-alpha))#I4
                TIT_out=Tthpc+(Icb-Ihpc)/Cp_t
                m_cb=(1.0-alpha)*m_hpc+m_fuel#m4
                Ptcb=Pthpc*(1.0-omega_cb)

                #作動点固定条件
                """HPT"""
                """高圧軸のエネルギー収支"""
                I_dot_tur_hpt=Icb-m_hpc*Cp_t*(Tthpc-Ttlpc)/(eta_mac*m_cb)
                T_dot_hpt=TIT_out+(I_dot_tur_hpt-Icb)/Cp_t
                I_tur_hpt=self.I45
                Tthpt=TIT_out+(I_tur_hpt-Icb)/Cp_t
                Pthpt=Ptcb*((T_dot_hpt/TIT_out-1.0)/eta_thp+1.0)**(gamma_t/(gamma_t-1.0))
                eps_ene=np.abs(((m_cb*I_dot_tur_hpt+m_hpc*beta1*Ihpc)-(m_cb+beta1*m_hpc)*I_tur_hpt)/(I_tur_hpt))#エネルギーの残差
                eps_ene_list.append(eps_ene)
                    
                HP_rate-=HP_rate_step#HP軸の修正回転数更新
                
            error_min=min(eps_ene_list)
            for i in range(len(eps_ene_list)):
                if error_min==eps_ene_list[i]:
                    index=i
            HP_rate=rate_max-index*HP_rate_step
            pai_hpc=1.0+(2.0-(MFP_rate*1.0/((HP_rate)))**b)**(1.0/a)*HP_rate**2*(pai_ref_hpc-1.0)#全圧比もとまる
            tau_hpc=(1.0+(pai_hpc**((gamma_c-1.0)/gamma_c)-1.0)/eta_chp)
            Pthpc=Pref*pai_hpc
            Tthpc=Tref*tau_hpc
            m_hpc=m_lpc#m3
            Ihpc=Cp_c*Tthpc#I3
                
            #燃焼器
            """CC"""
            m_fuel=self.m_dot_fuel#燃料流量
            Icb=(m_fuel*eta_cb*h+m_hpc*(1.0-alpha)*Ihpc)/(m_fuel+m_hpc*(1.0-alpha))#I4
            TIT_out=Tthpc+(Icb-Ihpc)/Cp_t
            m_cb=(1.0-alpha)*m_hpc+m_fuel#m4
            Ptcb=Pthpc*(1.0-omega_cb)

            #作動点固定条件
            """HPT"""
            """高圧軸のエネルギー収支"""
            I_dot_tur_hpt=Icb-m_hpc*Cp_t*(Tthpc-Ttlpc)/(eta_mac*m_cb)
            T_dot_hpt=TIT_out+(I_dot_tur_hpt-Icb)/Cp_t
            I_tur_hpt=self.I45
            Tthpt=TIT_out+(I_tur_hpt-Icb)/Cp_t
            Pthpt=Ptcb*((T_dot_hpt/TIT_out-1.0)/eta_thp+1.0)**(gamma_t/(gamma_t-1.0))
            eps_ene=np.abs(((m_cb*I_dot_tur_hpt+m_hpc*beta1*Ihpc)-(m_cb+beta1*m_hpc)*I_tur_hpt)/(I_tur_hpt))#エネルギーの残差
                
                    
            """LPT"""
            """低圧軸のエネルギー収支"""
            m_hpt=m_cb+beta1*m_hpc
            I_dot_tur_lpt=I_tur_hpt-(self.BPR/(1.0+self.BPR)*m_dot0*Cp_c*(Ttfan-Tref)+1.0/(1.0+self.BPR)*m_dot0*Cp_c*(Ttlpc-Tref))/(eta_mac*m_hpt)
            T_dot_lpt=Tthpt+(I_dot_tur_lpt-I_tur_hpt)/Cp_t
            Ptlpt=Pthpt*((T_dot_lpt/Tthpt-1.0)/eta_tlp+1.0)**(gamma_t/(gamma_t-1.0))
            I_tur_lpt=(m_hpt*I_dot_tur_lpt+m_hpc*beta2*Ihpc)/(m_hpt+beta2*m_hpc)#エネルギー収支
            Ttlpt=Tthpt+(I_tur_lpt-I_tur_hpt)/Cp_t
            pai_lpt=Ptlpt/Pref
            tau_lpt=Ttlpt/Tref
            m_lpt=m_hpt+beta2*m_hpc
            
            
            """Nozzle"""
            """ノズルの上流条件からノズル面積を計算"""
            Ptnoz=Ptlpt*(1.0-omega_core)
            Ttnoz=Ttlpt
            m_noz=m_lpt
            P_noz_ref=Pref/(1.0+(gamma_c-1.0)/2.0*M**2)**(gamma_c/(gamma_c-1.0))
            
            if Ptnoz/P_noz_ref<=((gamma_t+1.0)/2.0)**(gamma_t/(gamma_t-1.0)):
                Pnoz=P_noz_ref
                Mnoz=np.sqrt(2.0/(gamma_t-1.0)*((Ptnoz/Pnoz)**((gamma_t-1.0)/gamma_t)-1.0))
                Tnoz=Ttnoz/(1.0+(gamma_t-1.0)/2.0*Mnoz**2)
                Vnoz=Mnoz*np.sqrt(gamma_t*R*Tnoz)
                rou_noz=Pnoz/(R*Tnoz)
                Anoz=m_noz/(rou_noz*Vnoz)#ノズル面積
                
            else:
                Mnoz=1.0
                Pnoz=Ptnoz/((gamma_t+1.0)/2.0)**(gamma_t/(gamma_t-1.0))
                Tnoz=Ttnoz/(gamma_t+1.0)/2.0
                Vnoz=np.sqrt(gamma_t*R*Tnoz)
                rou_noz=Pnoz/(R*Tnoz)
                Anoz=m_noz/(rou_noz*Vnoz)#ノズル面積
                
            """ノズル面積が一致するか"""
            eps_area=np.abs((Anoz-self.A9)/self.A9)
            eps_area_list.append(eps_area)
            MFP_rate-=MFP_rate_step#ファン修正流量を更新
        
        
        """MFP_rate decision"""
        error_area_min=min(eps_area_list)
        for i in range(len(eps_area_list)):
            if error_area_min==eps_area_list[i]:
                index=i
        MFP_rate=MFP_rate_max-MFP_rate_step*index
        print('ans:','fan修正回転数',MFP_rate,'HP軸修正回転数割合',HP_rate)
        
        
        """決定した値で計算"""
        """fan"""
        pai_fan=1.0+(2.0-(MFP_rate*1.0/(LP_rate))**b)**(1.0/a)*LP_rate**2*(pai_ref_fan-1.0)#全圧比もとまる
        tau_fan=(1.0+(pai_fan**((gamma_c-1.0)/gamma_c)-1.0)/eta_fan)
        mfan=pai_fan/np.sqrt(tau_fan)*MFP_rate*m_fan_fix#ファンバイパス流量
        mcore=m_dot0-mfan#コア流量
            
        Ptfan=Pref*pai_fan#ファンの全圧
        Ttfan=Tref*tau_fan#ファンの全温
        Ifan=Cp_c*Ttfan#ファン出口のエンタルピー
            
        """LPC"""
        """LPCのマッハ数はM25"""
        m_lpc_fix=(self.m_dot25*np.sqrt(self.Tt25/Tref)/(self.Pt25/Pref))#修正流量
        MFP_ref_lpc=m_lpc_fix*np.sqrt(Tref)/(self.A25*Pref)
        pai_ref_lpc=self.Pt25/Pref
        pai_lpc=1.0+(2.0-(MFP_rate*1.0/((LP_rate)))**b)**(1.0/a)*LP_rate**2*(pai_ref_lpc-1.0)#全圧比もとまる
        tau_lpc=(1.0+(pai_lpc**((gamma_c-1.0)/gamma_c)-1.0)/eta_clp)
        m_lpc=mcore
        Ptlpc=Pref*pai_lpc
        Ttlpc=Tref*tau_lpc
            
            
        #loop2
        """HPC"""
        """HPCのマッハ数はM3"""
        m_hpc_fix=(self.m_dot3*np.sqrt(self.Tt3/Tref)/(self.Pt3/Pref))
        MFP_ref_hpc=m_hpc_fix*np.sqrt(Tref)/(self.A3*Pref)
        pai_ref_hpc=self.Pt3/Pref
        HP_rate=HP_rate#HP軸の修正回転数を仮定する
        pai_hpc=1.0+(2.0-(MFP_rate*1.0/((HP_rate)))**b)**(1.0/a)*HP_rate**2*(pai_ref_hpc-1.0)#全圧比もとまる
        tau_hpc=(1.0+(pai_hpc**((gamma_c-1.0)/gamma_c)-1.0)/eta_chp)
        Pthpc=Pref*pai_hpc
        Tthpc=Tref*tau_hpc
        m_hpc=m_lpc#m3
        Ihpc=Cp_c*Tthpc#I3
                
        #燃焼器
        """CC"""
        m_fuel=self.m_dot_fuel#燃料流量
        Icb=(m_fuel*eta_cb*h+m_hpc*(1.0-alpha)*Ihpc)/(m_fuel+m_hpc*(1.0-alpha))#I4
        TIT_out=Tthpc+(Icb-Ihpc)/Cp_t
        m_cb=(1.0-alpha)*m_hpc+m_fuel#m4
        Ptcb=Pthpc*(1.0-omega_cb)

        #作動点固定条件
        """HPT"""
        """高圧軸のエネルギー収支"""
        I_dot_tur_hpt=Icb-m_hpc*Cp_t*(Tthpc-Ttlpc)/(eta_mac*m_cb)
        T_dot_hpt=TIT_out+(I_dot_tur_hpt-Icb)/Cp_t
        I_tur_hpt=self.I45
        Tthpt=TIT_out+(I_tur_hpt-Icb)/Cp_t
        Pthpt=Ptcb*((T_dot_hpt/TIT_out-1.0)/eta_thp+1.0)**(gamma_t/(gamma_t-1.0))
                
                
                    
        """LPT"""
        """低圧軸のエネルギー収支"""
        m_hpt=m_cb+beta1*m_hpc
        I_dot_tur_lpt=I_tur_hpt-(self.BPR/(1.0+self.BPR)*m_dot0*Cp_c*(Ttfan-Tref)+1.0/(1.0+self.BPR)*m_dot0*Cp_c*(Ttlpc-Tref))/(eta_mac*m_hpt)
        T_dot_lpt=Tthpt+(I_dot_tur_lpt-I_tur_hpt)/Cp_t
        Ptlpt=Pthpt*((T_dot_lpt/Tthpt-1.0)/eta_tlp+1.0)**(gamma_t/(gamma_t-1.0))
        I_tur_lpt=(m_hpt*I_dot_tur_lpt+m_hpc*beta2*Ihpc)/(m_hpt+beta2*m_hpc)#エネルギー収支
        Ttlpt=Tthpt+(I_tur_lpt-I_tur_hpt)/Cp_t
        pai_lpt=Ptlpt/Pref
        tau_lpt=Ttlpt/Tref
        m_lpt=m_hpt+beta2*m_hpc
            
            
        """Nozzle"""
        """ノズルの上流条件からノズル面積を計算"""
        Ptnoz=Ptlpt*(1.0-omega_core)
        Ttnoz=Ttlpt
        m_noz=m_lpt
        P_noz_ref=Pref/(1.0+(gamma_c-1.0)/2.0*M**2)**(gamma_c/(gamma_c-1.0))
            
        if Ptnoz/P_noz_ref<=((gamma_t+1.0)/2.0)**(gamma_t/(gamma_t-1.0)):
            Pnoz=P_noz_ref
            Mnoz=np.sqrt(2.0/(gamma_t-1.0)*((Ptnoz/Pnoz)**((gamma_t-1.0)/gamma_t)-1.0))
            Tnoz=Ttnoz/(1.0+(gamma_t-1.0)/2.0*Mnoz**2)
            Vnoz=Mnoz*np.sqrt(gamma_t*R*Tnoz)
            rou_noz=Pnoz/(R*Tnoz)
            Anoz=m_noz/(rou_noz*Vnoz)#ノズル面積
                
        else:
            Mnoz=1.0
            Pnoz=Ptnoz/((gamma_t+1.0)/2.0)**(gamma_t/(gamma_t-1.0))
            Tnoz=Ttnoz/(gamma_t+1.0)/2.0
            Vnoz=np.sqrt(gamma_t*R*Tnoz)
            rou_noz=Pnoz/(R*Tnoz)
            Anoz=m_noz/(rou_noz*Vnoz)#ノズル面積
        
        """fan"""
        
        Mfan=np.sqrt(2/(gamma_c-1)*(Ttfan/Tref-1.0))
        Tfan=Ttfan/(1.0+(gamma_c-1.0)/2.0*Mfan**2)
        Pfan=P_noz_ref
        roufan=Pfan/(R*Tfan)
        Vfan=Mfan*np.sqrt(gamma_c*R*Tfan)
        Afan=mfan/(roufan*Vfan)
        
        
        
        ISP=(m_noz*Vnoz+mfan*Vfan-m_dot0*M*np.sqrt(gamma_c*R*Tref))/gc+Afan*(Pfan-P_noz_ref)+Anoz*(Pnoz-P_noz_ref)#比推力
        ISP=np.sqrt(ISP.real**2+ISP.imag**2)#虚数なので距離をとる
        print(ISP)
        SFC=m_fuel/ISP*3600*gc
        
        print(ISP,SFC.real,TIT_out.real)
        
        return ISP.real,SFC.real,TIT_out.real
        
            
            
                
             
        
    """設計点計算ループ用"""
    def cycle_1(self,BPR,FPR,OPR,TIT):
        pai_clp=np.sqrt(OPR)
        pai_chp=np.sqrt(OPR)
        #入口大気
        Pt0=P0*(1.0+(gamma_c-1.0)/2.0*Ma**2)**((gamma_c)/(gamma_c-1))
        Tt0=T0*(1.0+(gamma_c-1.0)/2.0*Ma**2)
        V0=np.sqrt(gamma_c*R*T0)

        #インテーク入口
        Pt2=Pt0*(1.0-omega_1)
        Tt2=Tt0
        m_dot2=m_dot0
        I2=Cp_c*Tt2

        #ファン出口(バイパス)
        Pt13=FPR*Pt2
        Tt13=Tt2*(1.0+(FPR**((gamma_c-1.0)/gamma_c)-1.0)/eta_fan)
        I13=I2+Cp_c*(Tt13-Tt2)
        m_dot13=BPR/(1.0+BPR)*m_dot0

        #ファンノズル
        Pt19=Pt13*(1.0-omega_np)
        Tt19=Tt13


        #ファン出口(コア)
        Pt21=FPR*Pt2
        Tt21=Tt2*(1.0+(FPR**((gamma_c-1.0)/gamma_c)-1.0)/eta_fan)
        I21=I2+Cp_c*(Tt21-Tt2)
        m_dot21=1.0/(1.0+BPR)*m_dot0
        M21=0.6#低圧圧縮機入口マッハ数

        #低圧圧縮機出口
        Pt25=Pt21*pai_clp
        Tt25=Tt21*(1.0+(pai_clp**((gamma_c-1.0)/gamma_c)-1.0)/eta_clp)
        I25=Cp_c*Tt25
        m_dot25=m_dot21
        M25=0.6#低圧圧縮機出口マッハ数

        #高圧圧縮機出口
        Pt3=Pt25*pai_chp
        Tt3=Tt25*(1.0+(pai_chp**((gamma_c-1.0)/gamma_c)-1.0)/eta_chp)
        I3=I25+Cp_c*(Tt3-Tt25)
        m_dot3=m_dot25
        M3=0.3#高圧縮機出口マッハ数

        #燃焼器出口
        Pt4=Pt3*(1.0-omega_cb)
        Tt4=TIT
        I4=I3+Cp_t*(Tt4-Tt3)
        m_dot_fuel=(m_dot3-alpha*m_dot3)*(I4-I3)/(eta_cb*h-I4)#エネルギー保存
        m_dot4=m_dot3-alpha*m_dot3+m_dot_fuel

        #高圧タービン出口
        I_d45=I4-m_dot3*(I3-I25)/(eta_mac*m_dot4)
        T_d45=Tt4-(I4-I_d45)/Cp_t
        Pt45=Pt4*((T_d45/Tt4-1.0)/eta_thp+1.0)**(gamma_t/(gamma_t-1.0))
        I45=(m_dot4*I_d45+m_dot3*beta1*I3)/(m_dot4+m_dot3*beta1)
        Tt45=Tt4-(I4-I45)/Cp_t
        m_dot45=m_dot4+beta1*m_dot3

        #低圧タービン出口
        I_d5=I45-(BPR/(1.0+BPR)*m_dot2*(I13-I2)+1.0/(1.0+BPR)*m_dot2*(I25-I2))/(eta_tlp*m_dot45)
        T_d5=Tt45-(I_d5-I45)/Cp_t
        Pt5=Pt45*((T_d5/Tt45-1.0)/eta_tlp+1.0)**(gamma_t/(gamma_t-1.0))
        I5=(m_dot45*I_d5+m_dot3*beta2*I3)/(m_dot45+m_dot3*beta2)
        Tt5=Tt45-(I5-I45)/Cp_t
        m_dot5=m_dot45+m_dot3*beta2

        #ノズル
        Pt9=Pt5*(1.0-omega_core)
        Tt9=Tt5
        m_dot9=m_dot5



        if Pt9/Pt0<=((gamma_t+1.0)/2.0)**(gamma_t/(gamma_t-1)):
            P9=P0
            M9=np.sqrt(2.0/(gamma_t-1.0)*((Pt9/Pt0)**((gamma_t-1.0)/gamma_t)-1.0))
            T9=Tt9/(1.0+(gamma_t-1.0)/2.0*M9**2)
            rou9=P9/(R*T9)
            V9=np.sqrt(gamma_t*R*T9)
            A9=m_dot9/(rou9*V9)
        else:
            M9=1.0
            T9=Tt9/(gamma_t+1.0)/2.0
            P9=Pt9/((gamma_t+1.0)/2.0)**(gamma_t/(gamma_t-1.0))
            rou9=P9/(R*T9)
            V9=np.sqrt(gamma_t*R*T9)
            A9=m_dot9/(rou9*V9)

        m_dot19=m_dot13
        P19=P0
        M19=np.sqrt(2.0/(gamma_c-1.0)*((Pt19/P19)**((gamma_c-1.0)/gamma_c)-1.0))
        T19=Tt19/(1.0+(gamma_c-1.0)/2.0*M19**2)
        V19=np.sqrt(gamma_c*R*T19)*M19
        rou19=P19/(R*T19)
        A19=m_dot19/(rou19*V19)




        #推力
        FN=(m_dot19*V19+m_dot9*V9-m_dot0*V0)/gc+A19*(P19-P0)+A9*(P9-P0)
        #SFC[kg/s]
        print(m_dot_fuel)
        SFC=m_dot_fuel/FN*3600*gc
        #比推力
        ISP=FN/m_dot0
        #入口出口圧力比
        tau=T9/T0


        return FN,SFC,ISP,tau,M9
    
    
    """ビッグデータ生成用設計点計算ループ"""
    def cycle_loop(self):
        #複数回回す計算
        FN,SFC,ISP,tau,Mj=[],[],[],[],[]
        for i1 in np.arange(3.0,5.0,1.0):
            for i2 in np.arange(1.0,1.5,0.1):
                for i3 in np.arange(10.0,50.0,10.0):
                    for i4 in np.arange(1200,1500,100):
                        BPR=i1
                        FPR=i2
                        OPR=i3
                        TIT=i4
                        fn,sfc,isp,t,mj=self.cycle_1(BPR,FPR,OPR,TIT)
                        FN.append(fn)
                        SFC.append(sfc)
                        ISP.append(isp)
                        tau.append(t)
                        Mj.append(mj)


        plt.figure()
        plt.scatter(FN,SFC)
        plt.xlabel('ISP')
        plt.ylabel('SFC')
        plt.show()

In [255]:
#parameters
params=[6.6,2.5,31.5,1380]#BPR,FPR,OPR,TITの順に格納
#飛行条件
M=0.80
H=10668
#性能計算インスタンス
jet=jetengine(params[0],params[1],params[2],params[3])
print('ISP,SFC,TIT:',jet.cycle(M,H))
print('ISP,SFC,ISP,tau,Mj:',jet.cycle_1(params[0],params[1],params[2],params[3]))

    

ans: fan修正回転数 1.15 HP軸修正回転数割合 0.51
83.29214690899516
83.29214690899516 0.3821612290843991 1330.524347041284
ISP,SFC,TIT: (83.29214690899516, 0.3821612290843991, 1330.524347041284)
0.0009022400577046464
ISP,SFC,ISP,tau,Mj: (45.775790615498344, 0.6953682024454838, 45.775790615498344, 1.3885644528984462, 1.0)


In [ ]:
#設計点外計算から騒音へ
